## Setup

The remainder of the notebook runs through the following steps

- Connect to Vantage
- Create DDLs
- Import Data


In [ ]:
%pip install teradataml==17.20.0.3 aoa==7.0.1 pandas==1.1.5

In [ ]:
from teradataml import create_context
import getpass
import logging
import sys
import urllib

logging.basicConfig(stream=sys.stdout, level=logging.INFO)


host = input("Host:")
username = input("Username:")
password = getpass.getpass("Password:")
database = input("Database (defaults to user):")

if not database:
    database = username


engine = create_context(host=host, 
                        username=username, 
                        password=urllib.parse.quote(password), 
                        logmech="TDNEGO",
                        database=database)

### Create DDLs

Create the following tables 

- aoa_statistics_metadata 
- aoa_byom_models
- pima_patient_predictions

`aoa_statistics_metadata` is used to store the profiling metadata for the features so that we can consistently compute the data drift and model drift statistics. This table can also be created via the CLI by executing 

```bash
aoa feature create-stats-table -e -m <statistics-metadata-db>.<statistics-metadata-table>
```

`pima_patient_predictions` is used for storing the predictions of the model scoring for the demo use case

In [ ]:
from aoa import create_features_stats_table
from teradataml import get_context

# Note: assuming we are using user database for training. If another database (e.g. datalab) is being used, please update.
# Also note, if a shared datalab is being used, only one user should execute the following DDL/DML commands
database = username

create_features_stats_table(f"{database}.aoa_statistics_metadata")

get_context().execute(f"""
CREATE MULTISET TABLE {database}.aoa_byom_models
    (
        model_version VARCHAR(255),
        model_id VARCHAR(255),
        model_type VARCHAR(255),
        project_id VARCHAR(255),
        deployed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        model BLOB(2097088000)
    )
    UNIQUE PRIMARY INDEX ( model_version );
""")

get_context().execute(f"""
CREATE MULTISET TABLE {database}.pima_patient_predictions
    (
        job_id VARCHAR(255),
        PatientId BIGINT,
        HasDiabetes BIGINT,
        json_report CLOB(1048544000) CHARACTER SET UNICODE
    )
    PRIMARY INDEX ( job_id );
""")

### Import Data

Create and import the data for the following two tables

- pima_patient_features
- pima_patient_diagnoses
- aoa_statistics_metadata

`pima_patient_features` contains the features related to the patients medical history.

`pima_patient_diagnoses` contains the diabetes diagnostic results for the patients.

`aoa_statistics_metadata` contains the feature statistics metadata for the `pima_patient_features` and `pima_patient_diagnoses`

Note the `pima_patient_feature` can be populated via the CLI by executing 

Compute the statistics metadata for the continuous variables
```bash
aoa feature compute-stats \
        -s <feature-db>.<feature-data> \
        -m <statistics-metadata-db>.<statistics-metadata-table> \
        -t continuous -c numtimesprg,plglcconc,bloodp,skinthick,twohourserins,bmi,dipedfunc,age
```

Compute the statistics metadata for the categorical variables
```bash
aoa feature compute-stats \
        -s <feature-db>.<feature-data> \
        -m <statistics-metadata-db>.<statistics-metadata-table> \
        -t categorical -c hasdiabetes
```

In [ ]:
from teradataml import copy_to_sql, DataFrame
from teradatasqlalchemy.types import *
import pandas as pd

df = pd.read_csv("data/pima_patient_features.csv")
copy_to_sql(df=df, 
            table_name="pima_patient_features",     
            schema_name=database,
            primary_index="PatientId", 
            if_exists="replace",  
            types={
                "PatientId": INTEGER,
                "NumTimesPrg": INTEGER, 
                "PlGlcConc": INTEGER,
                "BloodP": INTEGER,
                "SkinThick": INTEGER,
                "TwoHourSerIns": INTEGER,
                "BMI": FLOAT,
                "DiPedFunc": FLOAT,
                "Age": INTEGER
            })

df = pd.read_csv("data/pima_patient_diagnoses.csv")
copy_to_sql(df=df, 
            table_name="pima_patient_diagnoses",     
            schema_name=database,
            primary_index="PatientId", 
            if_exists="replace",  
            types={
                "PatientId": INTEGER,
                "HasDiabetes": INTEGER
            })

# we can compute this from the CLI also - but lets import pre-computed for now.
df = pd.read_csv("data/aoa_statistics_metadata.csv")
copy_to_sql(df=df, 
            table_name="aoa_statistics_metadata",     
            schema_name=database,
            if_exists="append")


## ModelOps UI

#### Add Project

- create project
   - Details
      - Name: Demo {your-name}
      - Description: ModelOps Demo
      - Group: {your-name}
      - Path: https://github.com/Teradata/modelops-demo-models 
      - Credentials: No Credentials
      - Branch: master
      - Save And Continue
   - Service Connection
      - Skip for now
   - Personal Connection
      - Name: Vantage Personal {your-name}
      - Description: Vantage Demo Env
      - Host: {your-host}
      - Database: {your-db}
      - VAL Database: {your-val-db}
      - BYOM Database: (your-byom-db}
      - Login Mech: TDNEGO
      - Username/Password
      
    
#### Add Datasets

- create dataset template
  - Catalog
     - Name: PIMA
     - Description: PIMA Diabetes
     - Feature Catalog: Vantage
     - Database: {your-db}
     - Table: aoa_statistics_metadata
  - Features
     - Query: `SELECT * FROM {your-db}.pima_patient_features`
     - Entity Key: PatientId
     - Features: NumTimesPrg, PlGlcConc, BloodP, SkinThick, TwoHourSerIns, BMI, DiPedFunc, Age
  - Entity & Target
     - Query: `SELECT * FROM {your-db}.pima_patient_diagnoses`
     - Entity Key: PatientId
     - Target: HasDiabetes
  - Predictions
     - Database: {your-db}
     - Table: pima_patient_predictions
     - Entity Selection: `SELECT * FROM pima_patient_features WHERE patientid MOD 5 = 0`
     
    
- create training dataset
   - Basic
      - Name: Train
      - Description: Training dataset
      - Scope: Training
   - Entity & Target
      - Query: `SELECT * FROM {your-db}.pima_patient_diagnoses WHERE patientid MOD 5 <> 0`
   
- create evaluation dataset
   - Basic
      - Name: Evaluate
      - Description: Evaluation dataset
      - Scope: Evaluation
   - Entity & Target
      - Query: `SELECT * FROM {your-db}.pima_patient_diagnoses WHERE patientid MOD 5 = 0`
    

#### Model Lifecycle

- Python Diabetes Prediction
   - Train
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Deployments/executions
   - Retire
- R Diabetes Prediction
   - Train
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Deployments/executions
   - Retire
- BYOM Diabetes Prediction
   - Run BYOM Notebook 
   - Define BYOM Model 
   - Import Version
   - Evaluate
   - Review evaluation report
   - Approve 
   - Deploy 
   - Deployments/executions
   - Retire

#### View Predictions

In the UI, select a deployment from the deployments left hand navigation. Go to the Jobs tab and on the right hand side for each job execution, you can select "View Predictions". This will show you a sample of the predictions for that particular job execution.

Note, your predictions table must have a `job_id` column which matches to the execution job id. If using BYOM, this is done automatically. For you own `scoring.py`, checkout the demo models.

## CLI 


```bash
pip install aoa>=7.0.0rc3
```

##### Copy CLI Config

```
Copy the CLI config from ModelOps UI -> Session Details -> CLI config
```

##### Add Dataset Connection

```bash
aoa connection add
```

##### List Feature Metadata

```bash
aoa feature list-stats -m {your-db}.aoa_feature_metadata
```

##### Clone Project

```bash
aoa clone 
```

```bash
cd modelops-demo-models
```

##### Install Model Dependencies

```bash
pip install -r model_definitions/python-diabetes/model_modules/requirements.txt
```

##### Train Model

```bash
aoa run
```

##### Add Model

```bash
aoa add
```
